# Grid

In [33]:
"""
This is the graph module. It contains a minimalistic Graph class.
"""

class Graph:
    """
    A class representing undirected graphs as adjacency lists. 

    Attributes: 
    -----------
    nodes: NodeType
        A list of nodes. Nodes can be of any immutable type, e.g., integer, float, or string.
        We will usually use a list of integers 1, ..., n.
    graph: dict
        A dictionnary that contains the adjacency list of each node in the form
        graph[node] = [neighbor1, neighbor2, ...]
    nb_nodes: int
        The number of nodes.
    nb_edges: int
        The number of edges. 
    edges: list[tuple[NodeType, NodeType]]
        The list of all edges
    """

    def __init__(self, nodes=[]):
        """
        Initializes the graph with a set of nodes, and no edges. 

        Parameters: 
        -----------
        nodes: list, optional
            A list of nodes. Default is empty.
        """
        self.nodes = nodes
        self.graph = dict([(n, []) for n in nodes])
        self.nb_nodes = len(nodes)
        self.nb_edges = 0
        self.edges = []
        
    def __str__(self):
        """
        Prints the graph as a list of neighbors for each node (one per line)
        """
        if not self.graph:
            output = "The graph is empty"            
        else:
            output = f"The graph has {self.nb_nodes} nodes and {self.nb_edges} edges.\n"
            for source, destination in self.graph.items():
                output += f"{source}-->{destination}\n"
        return output

    def __repr__(self): 
        """
        Returns a representation of the graph with number of nodes and edges.
        """
        return f"<graph.Graph: nb_nodes={self.nb_nodes}, nb_edges={self.nb_edges}>"

    def add_edge(self, node1, node2):
        """
        Adds an edge to the graph. Graphs are not oriented, hence an edge is added to the adjacency list of both end nodes. 
        When adding an edge between two nodes, if one of the ones does not exist it is added to the list of nodes.

        Parameters: 
        -----------
        node1: NodeType
            First end (node) of the edge
        node2: NodeType
            Second end (node) of the edge
        """
        if node1 not in self.graph:
            self.graph[node1] = []
            self.nb_nodes += 1
            self.nodes.append(node1)
        if node2 not in self.graph:
            self.graph[node2] = []
            self.nb_nodes += 1
            self.nodes.append(node2)

        self.graph[node1].append(node2)
        self.graph[node2].append(node1)
        self.nb_edges += 1
        self.edges.append((node1, node2))

    def bfs(self, node_1, node_2): 
        """
        Finds a shortest path from src to dst by BFS.  

        Parameters: 
        -----------
        src: NodeType
            The source node.
        dst: NodeType
            The destination node.

        Output:
        -------
        path: list[NodeType] | None
            The shortest path from src to dst. Returns None if dst is not reachable from src
        """ 
        def find_neighbors(edges, node):
            edges_out = [edge for edge in edges if edge[0] == node]
            neighbors = [edge[1] for edge in edges_out]
            return neighbors
        graph = self
        unoriented_edges = list(set(graph.edges + [(edge[1], edge[0]) for edge in graph.edges]))
        list_paths = [edge for edge in unoriented_edges if edge[0] == node_1]
        if node_1 == node_2:
            return []
        found = False
        while not found:
            new_paths = []
            for path in list_paths:
                if path[-1] == node_2:
                    # Found it! Return the distance and the path
                    return path
                else:
                    path_neighbors = find_neighbors(unoriented_edges, path[-1])
                    new_paths += [path + (new_node,) for new_node in path_neighbors]
            list_paths = new_paths


    @classmethod
    def graph_from_file(cls, file_name):
        """
        Reads a text file and returns the graph as an object of the Graph class.

        The file should have the following format: 
            The first line of the file is 'n m'
            The next m lines have 'node1 node2'
        The nodes (node1, node2) should be named 1..n

        Parameters: 
        -----------
        file_name: str
            The name of the file

        Outputs: 
        -----------
        graph: Graph
            An object of the class Graph with the graph from file_name.
        """
        with open(file_name, "r") as file:
            n, m = map(int, file.readline().split())
            graph = Graph(range(1, n+1))
            for _ in range(m):
                edge = list(map(int, file.readline().split()))
                if len(edge) == 2:
                    node1, node2 = edge
                    graph.add_edge(node1, node2) # will add dist=1 by default
                else:
                    raise Exception("Format incorrect")
        return graph


In [34]:
import random
import pygame
class GridVisualizer:
    def __init__(self, grid):
        """
        Initializes the grid visualizer.
        
        grid: 2D list representing the grid state
        cell_size: Size of each cell in pixels
        grid_color: Color of the grid lines (R, G, B)
        """
        self.grid = grid
        self.m = len(grid)
        self.n = len(grid[0])
        self.cell_size = 50
        self.grid_color = (0,0,0)
        

        self.width = self.n * self.cell_size
        self.height = self.m * self.cell_size
        

        pygame.init()
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Grid Representation")
        

        self.background_color = (255, 255, 255)
    
    def draw_grid(self):
        """
        Draws the grid on the window, filling each cell with its value.
        """
        self.screen.fill(self.background_color)
        
        # Initialize font; you might need to adjust the size for visibility
        font = pygame.font.Font(None, 24)  # None uses the default font, 24 is the font size
        
        for i in range(self.m):
            for j in range(self.n):
                cell_value = self.grid[i][j]
                

                rect = pygame.Rect(j * self.cell_size, i * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(self.screen, self.grid_color, rect, 1)
                

                text = font.render(str(cell_value), True, (0, 0, 0))

                text_pos = text.get_rect(center=rect.center)
                

                self.screen.blit(text, text_pos)
                
        pygame.display.flip()

    def run(self):
        """
        Main loop for the grid visualizer.
        """
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    
            self.draw_grid()
            
        pygame.quit()

class Grid():
    """
    A class representing the grid from the swap puzzle. It supports rectangular grids. 

    Attributes: 
    -----------
    m: int
        Number of lines in the grid
    n: int
        Number of columns in the grid
    state: list[list[int]]
        The state of the grid, a list of list such that state[i][j] is the number in the cell (i, j), i.e., in the i-th line and j-th column. 
        Note: lines are numbered 0..m and columns are numbered 0..n.
    """
    
    def __init__(self, m, n, initial_state = []):
        """
        Initializes the grid.

        Parameters: 
        -----------
        m: int
            Number of lines in the grid
        n: int
            Number of columns in the grid
        initial_state: list[list[int]]
            The intiail state of the grid. Default is empty (then the grid is created sorted).
        """
        self.m = m
        self.n = n
        if not initial_state:
            initial_state = [list(range(i*n+1, (i+1)*n+1)) for i in range(m)]            
        self.state = initial_state


    def __str__(self):
        self.visualizer = GridVisualizer(self.state)
        self.visualizer.run()


    def __repr__(self): 
        """
        Returns a representation of the grid with number of rows and columns.
        """
        return f"<grid.Grid: m={self.m}, n={self.n}>"
    
    def make_hashable(self):
        """Tuples are hashable, so just convert the graph to a tuple"""
        list_tuples = [tuple(self.state[i]) for i in range(self.m)]
        tuple_tuples = tuple(list_tuples)
        return tuple_tuples
    
    def is_sorted(self):
        """
        Checks is the current state of the grid is sorted and returns the answer as a boolean.
        """
        for i in range(self.m):
            for j in range(self.n - 1):
                if self.state[i][j] > self.state[i][j+1]:
                    return False
        for i in range(0, self.m - 1):
            if self.state[i][-1] > self.state[i+1][0]:
                return False
        return True

    
    def swap(self, cell1, cell2):

        """
        Implements the swap operation between two cells. Raises an exception if the swap is not allowed.

        Parameters: 
        -----------
        cell1, cell2: tuple[int]
            The two cells to swap. They must be in the format (i, j) where i is the line and j the column number of the cell. 
        """
        i1, j1 = cell1
        i2, j2 = cell2
        if (abs(i1-i2)==0 and abs(j1-j2)==1) or (abs(i1-i2)==1 and abs(j1-j2)==0):
             self.state[i1][j1], self.state[i2][j2] = self.state[i2][j2], self.state[i1][j1]

    def swap_seq(self, cell_pair_list):
        """
        Executes a sequence of swaps. 

        Parameters: 
        -----------
        cell_pair_list: list[tuple[tuple[int]]]
            List of swaps, each swap being a tuple of two cells (each cell being a tuple of integers). 
            So the format should be [((i1, j1), (i2, j2)), ((i1', j1'), (i2', j2')), ...].
        """
        for cell_1, cell_2 in cell_pair_list:
            self.swap(cell_1, cell_2)
        
    def grids_graph(self):
        m = self.m
        n = self.n
        from itertools import permutations
        all_permutations = permutations(range(1, n*m + 1))
        tuples = []
        for permutation in all_permutations:
            perm_lines = [permutation[n*i:n*(i+1)] for i in range(m)]
            perm_tuple = tuple(tuple(line) for line in perm_lines)
            tuples.append(perm_tuple)
        return tuples

    def all_edges(self):
        if not hasattr(self, "path_graph"):
            self.path_graph = Graph(self.grids_graph())
        nodes = self.path_graph.nodes
        from copy import deepcopy
        m = len(nodes[0])
        n = len(nodes[0][0])
        list_transpositions = [((i,j),(i,j+1)) for i in range(m) for j in range(n-1)] + [((i,j),(i+1,j)) for i in range(m-1) for j in range(n)]
        list_edges = set()
        for node in nodes:
            # Find all possible neighbors
            node_list = [list(line) for line in node]
            for transposition in list_transpositions:
                target = deepcopy(node_list)
                ((i_0, j_0), (i_1, j_1)) = transposition
                target[i_0][j_0], target[i_1][j_1] = target[i_1][j_1], target[i_0][j_0]
                target_tuple = tuple(tuple(line) for line in target)
                list_edges.add((node, target_tuple))
                list_edges.add((target_tuple, node))
        return list(list_edges)
    def find_best_path(self):
        if not hasattr(self, "path_graph"):
            path_graph_nodes = self.grids_graph()
            self.path_graph = Graph(path_graph_nodes)
            self.path_graph.edges = self.all_edges()
            # We do not completely fulfill the graph attributes because it is not useful
        n = self.n
        m = self.m
        solved = tuple(tuple(range(n*i+1,n*(i+1)+1)) for i in range(m))
        current = tuple(tuple(line) for line in self.state)
        return self.path_graph.bfs(current, solved)
    

    @classmethod
    def grid_from_file(cls, file_name): 
        """
        Creates a grid object from class Grid, initialized with the information from the file file_name.
        
        Parameters: 
        -----------
        file_name: str
            Name of the file to load. The file must be of the format: 
            - first line contains "m n" 
            - next m lines contain n integers that represent the state of the corresponding cell

        Output: 
        -------
        grid: Grid
            The grid
        """
        with open(file_name, "r") as file:
            m, n = map(int, file.readline().split())
            initial_state = [[] for i_line in range(m)]
            for i_line in range(m):
                line_state = list(map(int, file.readline().split()))
                if len(line_state) != n: 
                    raise Exception("Format incorrect")
                initial_state[i_line] = line_state
            grid = Grid(m, n, initial_state)
        return grid


In [35]:
class Solver(): 
    """
    A solver class, to be implemented.
    """
    def __init__(self, grid):
        self.grid = grid



    def get_solution(self):
        """
        Solves the grid and returns the sequence of swaps at the format 
        [((i1, j1), (i2, j2)), ((i1', j1'), (i2', j2')), ...]. 
        """
        big_list_moves = []


        # 1. Find the element
        def find_element(element, grid):
            for i in range(grid.m):
                for j in range(grid.n):
                    if grid.state[i][j] == element:
                        return (i, j)

        # 2. For each line, give it the elements it should contain
        def align_element_in_line(grid, line_index, current_column, shift, list_moves = []):
            """We shift the column of an element in some line recursively
            Input:
            - line: the list of elements in the line. Is obtained with grid.state[i]
            - current_position: index of the element to be shifted in the list
            - shift: the shift to be applied
            - list_moves: should be left blank. Used as an argument for the recursion
            Output:
            - The list of moves performed.
            """
            line = grid.state[line_index]
            if shift == 0:
                return list_moves
            else:
                if shift >= 1:
                    
                    line[current_column], line[current_column + 1] = line[current_column + 1], line[current_column]
                    list_moves.append(((line_index, current_column),(line_index, current_column + 1)))
                    return align_element_in_line(grid, line_index, current_column + 1, shift - 1, list_moves)
                if shift <= -1:
                    line[current_column-1], line[current_column] = line[current_column], line[current_column-1]
                    list_moves.append(((line_index, current_column),(line_index, current_column - 1)))
                    return align_element_in_line(grid, line, current_column - 1, shift + 1, list_moves)

        def get_elements_for_line(grid, line_index,):
            """Brings the elements [line*self.grid.n, ... (line+1)*self.grid.n - 1] on the line line_index."""
            big_list_moves = []
            # The line has elements [line*self.grid.n, ... (line+1)*self.grid.n - 1]
            list_elements = list(range(line_index*grid.n+1, (line_index+1)*grid.n+1)) # The numbers start at 1
            print(f"{list_elements=}")
            #  All elements that are supposed to be on this line/
            for element in list_elements:
                # Find where it is.
                l_good, c_good = find_element(element, grid)
                if l_good != line_index:
                    # Look for a column in line line_index that contains a bad element
                    for column, element in enumerate(grid.state[line_index]):
                        if not element in list_elements:
                            c_bad = column
                            break # Get out of the for loop when we find one.
                    # Now we have the column of a bad element in line_index. Move the element at position i, j to position i, column_bad_element.
                    big_list_moves += align_element_in_line(grid, l_good, c_good, c_bad-c_good)
                    if l_good > line_index:
                        sign = -1
                    else:
                        sign = 1
                    list_moves = [((l,c_bad),(l+sign, c_bad)) for l in range(l_good, line_index, sign)]
                    grid.swap_seq(list_moves)
                    big_list_moves += list_moves
            return big_list_moves

        # Now put all elements in the right lines:
        for line_index in range(self.grid.m):
            big_list_moves += get_elements_for_line(self.grid, line_index)
        
        def sort_all_lines(grid):
            list_moves = []
            for i in range(self.grid.m):
                for j in range(self.grid.n-1):
                    if grid.state[i][j] > grid.state[i][j+1]:
                        list_moves.append(((i,j),(i,j+1)))
                        grid.state[i][j], grid.state[i][j+1] = grid.state[i][j+1], grid.state[i][j]
            return list_moves

        while not self.grid.is_sorted():
            big_list_moves += sort_all_lines(self.grid)

        return big_list_moves

In [36]:
grille_init = [[1,3],[4,2]]
nouvelle_grille = Grid(len(grille_init),len(grille_init[0]), grille_init)

In [37]:
d = nouvelle_grille.find_best_path()

In [38]:
d

(((1, 3), (4, 2)), ((1, 2), (4, 3)), ((1, 2), (3, 4)))

In [39]:
nouveau_solver = Solver(nouvelle_grille)

In [40]:
nouveau_solver.get_solution(
)

list_elements=[1, 2]
list_elements=[3, 4]


[((1, 1), (0, 1)), ((1, 0), (1, 1))]

In [41]:
print(nouveau_solver.grid)

TypeError: __str__ returned non-string (type NoneType)

# TP2

In [43]:
!pwd

/Users/milamarsot/Documents/github/projet-python/swap_puzzle


In [44]:
# BFS

graph = Graph.graph_from_file("../input/graph1.in")


In [55]:
print(graph)

The graph has 20 nodes and 100 edges.
1-->[15, 11, 17, 8, 5, 16, 13, 20, 4, 13, 9, 10, 2, 16, 18, 5]
2-->[14, 19, 13, 15, 11, 16, 1]
3-->[6, 18, 18, 7, 10, 10, 15, 6, 12, 11]
4-->[15, 9, 16, 12, 10, 8, 1, 18, 17, 6, 10]
5-->[12, 16, 1, 12, 12, 11, 6, 8, 17, 15, 16, 1, 11]
6-->[3, 19, 18, 7, 13, 5, 8, 20, 8, 13, 9, 4, 3]
7-->[3, 12, 6]
8-->[12, 15, 1, 15, 15, 19, 4, 6, 6, 5, 10]
9-->[4, 17, 15, 1, 6]
10-->[18, 20, 17, 19, 3, 4, 14, 15, 3, 12, 20, 1, 8, 4, 18, 11]
11-->[1, 5, 20, 20, 16, 2, 16, 3, 10, 5]
12-->[17, 8, 5, 5, 5, 7, 4, 14, 15, 10, 3]
13-->[1, 6, 2, 6, 1, 20]
14-->[16, 2, 12, 10, 19, 20, 17]
15-->[1, 18, 4, 8, 16, 9, 8, 8, 12, 20, 10, 17, 3, 2, 20, 5]
16-->[19, 14, 15, 5, 1, 4, 19, 11, 2, 11, 1, 5]
17-->[12, 1, 9, 10, 15, 4, 5, 14]
18-->[15, 6, 10, 3, 3, 4, 1, 10]
19-->[16, 6, 10, 8, 2, 16, 14]
20-->[10, 15, 6, 1, 11, 11, 15, 10, 13, 14]



In [49]:
def find_neighbors(edges, node):
    edges_out = [edge for edge in edges if edge[0] == node]
    neighbors = [edge[1] for edge in edges_out]
    return neighbors
def bfs(graph, node_1, node_2):
    unoriented_edges = graph.edges + [(edge[1], edge[0]) for edge in graph.edges]
    list_paths = [edge for edge in unoriented_edges if edge[0] == node_1]
    if node_1 == node_2:
        return []
    found = False
    while not found:
        new_paths = []
        for path in list_paths:
            if path[-1] == node_2:
                # Found it! Return the distance and the path
                return path
            else:
                path_neighbors = find_neighbors(unoriented_edges, path[-1])
                new_paths += [path + (new_node,) for new_node in path_neighbors]
        list_paths = new_paths
    

In [53]:
def find_neighbors(edges, node):
    edges_out = [edge for edge in edges if edge[0] == node]
    neighbors = [edge[1] for edge in edges_out]
    return neighbors
def bfs(graph, node_1, node_2):
    unoriented_edges = graph.edges + [(edge[1], edge[0]) for edge in graph.edges]
    list_paths = [edge for edge in unoriented_edges if edge[0] == node_1]
    if node_1 == node_2:
        return []
    found = False
    while not found:
        new_paths = []
        for path in list_paths:
            if path[-1] == node_2:
                # Found it! Return the distance and the path
                return path
            else:
                path_neighbors = find_neighbors(unoriented_edges, path[-1])
                new_paths += [path + (new_node,) for new_node in path_neighbors]
        list_paths = new_paths
def grids_graph(m, n):
    from itertools import permutations
    all_permutations = permutations(range(1, n*m + 1))
    tuples = []
    for permutation in all_permutations:
        perm_lines = [permutation[n*i:n*(i+1)] for i in range(m)]
        perm_tuple = tuple(tuple(line) for line in perm_lines)
        tuples.append(perm_tuple)
    return tuples

def all_edges(nodes):
    from copy import deepcopy
    m = len(nodes[0])
    n = len(nodes[0][0])
    list_transpositions = [((i,j),(i,j+1)) for i in range(m) for j in range(n-1)] + [((i,j),(i+1,j)) for i in range(m-1) for j in range(n)]
    list_edges = set()
    for node in nodes:
        # Find all possible neighbors
        node_list = [list(line) for line in node]
        for transposition in list_transpositions:
            target = deepcopy(node_list)
            ((i_0, j_0), (i_1, j_1)) = transposition
            target[i_0][j_0], target[i_1][j_1] = target[i_1][j_1], target[i_0][j_0]
            target_tuple = tuple(tuple(line) for line in target)
            list_edges.add((node, target_tuple))
            list_edges.add((target_tuple, node))
    return list_edges

In [54]:
bfs(graph, 2, 4)


(2, 1, 4)

In [ ]:
all_grids = grids_graph(2,2)

In [ ]:

def all_edges(nodes):
    from copy import deepcopy
    m = len(nodes[0])
    n = len(nodes[0][0])
    list_transpositions = [((i,j),(i,j+1)) for i in range(m) for j in range(n-1)] + [((i,j),(i+1,j)) for i in range(m-1) for j in range(n)]
    list_edges = set()
    for node in nodes:
        # Find all possible neighbors
        node_list = [list(line) for line in node]
        for transposition in list_transpositions:
            target = deepcopy(node_list)
            ((i_0, j_0), (i_1, j_1)) = transposition
            target[i_0][j_0], target[i_1][j_1] = target[i_1][j_1], target[i_0][j_0]
            target_tuple = tuple(tuple(line) for line in target)
            list_edges.add((node, target_tuple))
            list_edges.add((target_tuple, node))
    return list_edges
    


In [ ]:
len(all_edges(all_grids))

96